In [1]:
import pandas as pd
import os
import requests, zipfile, io
import numpy as np


# Managers

In [2]:
managers = pd.read_csv("data/ipo_managers.csv", header=[0,1])


In [3]:
managers.columns = ['_'.join(col) for col in managers.columns.values]


In [4]:
managers.columns = managers.columns.str.replace(' ','_').str.lower()

In [5]:
managers 

,trade_date,unnamed:_1_level_0_issuer,unnamed:_2_level_0_symbol,lead/joint-lead__managers,offer_price,opening_price,1st_day_close,1st_day_%_px_chng_,$_change_opening,$_change_close,star_ratings,unnamed:_11_level_0_performed
0,1/17/20,I-Mab,IMAB,Jefferies/ CICC,$14.00,$14.75,$12.75,-8.93%,$0.75,($1.25),2,NaN
1,1/17/20,LIZHI,LIZI,Credit Suisse/ Citigroup,$11.00,$11.03,$11.63,5.73%,$0.03,$0.63,1,NaN
2,1/17/20,Phoenix Tree Holdings Ltd.,DNK,Citigroup/ Credit Suisse/ J.P. Morgan,$13.50,$13.50,$13.50,0.00%,$0.00,$0.00,1,NaN
3,1/17/20,Velocity Financial,VEL,Wells Fargo Securities/ Citigroup/ JMP Securities,$13.00,$13.85,$13.51,3.92%,$0.85,$0.51,1,NaN
4,1/24/20,Gores Holdings IV,GHIVU,Deutsche Bank Securities,$10.00,$10.52,$10.45,4.50%,$0.52,$0.45,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
3628,12/8/00,Specialty Laboratories,SP,Merrill Lynch,$16.00,$21.94,$23.31,45.70%,$5.94,$7.31,3,NaN
3629,12/8/00,W.P. Stewart & Co.,WPL,Goldman Sachs/Lazard,$20.00,$22.06,$24.50,22.50%,$2.06,$4.50,2,NaN
3630,12/11/00,Gemplus,GEMP,Credit Suisse First Boston,$10.69,$12.00,$13.38,25.12%,$1.31,$2.69,1,NaN
3631,12/12/00,GenVec,GNVC,J.P. Morgan,$9.50,$10.56,$9.88,3.95%,$1.06,$0.38,2,NaN


In [6]:
managers.rename(columns={
    'unnamed:_1_level_0_issuer':'company_name', 
    'unnamed:_2_level_0_symbol': 'ticker',
    'lead/joint-lead__managers': 'all_managers',
    'opening_price': 'open',
    '1st_day_close': 'close',
    'offer_price': 'offer',

    'unnamed:_11_level_0_performed': 'performed'},
                inplace=True)

In [7]:
managers.drop(managers.columns[np.r_[7:12]],inplace=True,axis=1)

In [8]:
managers.drop_duplicates(inplace=True)

In [9]:
managers= managers.groupby(['ticker']).max().reset_index()

**Data Formatting**

In [10]:
def format_prices(df,col):
    df[col] = df[col].str.strip('$\n\t').str.replace(',','').astype(float)
    return df[col]

cols = ['close', 'open','offer']

for col in cols:
    format_prices(managers,col)


In [11]:
managers['trade_date'] = pd.to_datetime(managers['trade_date'], errors = 'coerce')


In [12]:
# turn multi-valued managers cells into multi-row, single-valued --> -->
managers_columns = list(managers.columns)
managers_columns.remove('all_managers')
managers_columns

['ticker', 'trade_date', 'company_name', 'offer', 'open', 'close']

In [13]:
managers.dropna(subset=['all_managers'], inplace=True)
managers['all_managers'] = managers['all_managers'].map(lambda x: x.replace("/"," ,"))

managers['all_managers'] = managers['all_managers'].map(lambda x: x.split(","))
managers['all_managers']

0       [J.P. Morgan ,  Credit Suisse ,  Goldman Sachs...
1                                      [Rodman & Renshaw]
2                             [Citigroup ,  Jefferies​ ​]
3                        [William Blair ,  Raymond James]
4                                          [Bear Stearns]
                              ...                        
3538                                     [Morgan Stanley]
3539    [Citigroup ,  Barclays ,  Wells Fargo Securiti...
3540                         [Jefferies ,  Piper Jaffray]
3541    [Citigroup , Goldman Sachs , JP Morgan , Banc ...
3542    [BofA Merrill Lynch , Goldman Sachs , J.P. Mor...
Name: all_managers, Length: 3543, dtype: object

In [14]:
managers_melt = managers.all_managers.apply(pd.Series) \
    .merge(managers, right_index = True, left_index = True) \
    .drop(["all_managers"], axis = 1) \
    .melt(id_vars = managers_columns , value_name = "managers") 


In [15]:
managers_melt.dropna(subset=['managers'], inplace=True)

In [16]:
managers_melt.drop(managers_melt.columns[np.r_[6]],inplace=True,axis=1)

In [17]:
managers_melt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7956 entries, 0 to 52096
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ticker        7956 non-null   object        
 1   trade_date    7952 non-null   datetime64[ns]
 2   company_name  7956 non-null   object        
 3   offer         7956 non-null   float64       
 4   open          7956 non-null   float64       
 5   close         7956 non-null   float64       
 6   managers      7956 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 497.2+ KB


In [18]:
managers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3543 entries, 0 to 3542
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ticker        3543 non-null   object        
 1   trade_date    3542 non-null   datetime64[ns]
 2   company_name  3543 non-null   object        
 3   all_managers  3543 non-null   object        
 4   offer         3543 non-null   float64       
 5   open          3543 non-null   float64       
 6   close         3543 non-null   float64       
dtypes: datetime64[ns](1), float64(3), object(3)
memory usage: 350.5+ KB


# Save Collected Data 

In [19]:
import joblib 
joblib.dump(managers_melt,"managers_melt.pkl")
joblib.dump(managers,"managers_all.pkl")


['managers_all.pkl']

In [20]:
ls


1_collect_clean_AGE.ipynb       data/
2_collect_clean_MANAGERS.ipynb  ipo_180.pkl
3a_collect_SEC.ipynb            managers_all.pkl
3b_clean_SEC.ipynb              managers_melt.pkl
4a_collect_YF1.ipynb            sec_10k.pkl
4b_clean_YF1.ipynb              sec_10k_raw.pkl
5a_collect_YF2.ipynb            yf1.pkl
5b_clean_YF2.ipynb              yf1_raw.pkl
age.pkl                         yf2.pkl
crowdfunding.pkl
